In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [ ]:
import datetime
import pandas as pd

In [ ]:
import re

date_pattern = re.compile('([0-9]+)/([0-9]+)/([0-9]+)')
time_pattern = re.compile('([0-9]+):([0-9]+)')
mothermilk_pattern_right = re.compile('.*右 ([0-9]+)分')
mothermilk_pattern_left = re.compile('.*左 ([0-9]+)分')
milk_pattern = re.compile('.*ミルク ([0-9]+)ml')
expressedmilk_pattern = re.compile('.*搾母乳 ([0-9]+)ml')
small_pattern = re.compile('.*おしっこ')
big_pattern = re.compile('.*うんち')
sleep_pattern_in = re.compile('.*寝る')
sleep_pattern_out = re.compile('.*起きる')

In [ ]:
target = '202010'

In [ ]:
df = pd.DataFrame(columns=['date', 'time', 'mothermilk', 'milk', 'expressedmilk', 'small', 'big', 'sleep'])

with open(f'data/raw/{target}.txt', 'r', encoding='UTF-8') as f:
    lines = f.readlines()
    for l in lines:
        date_result = date_pattern.match(l)
        if date_result:
            date = datetime.date(int(date_result.group(1)), int(date_result.group(2)), int(date_result.group(3)))
            print(date)

        time_result = time_pattern.match(l)
        if time_result:
            time = datetime.time(int(time_result.group(1)), int(time_result.group(2)))
        
        mothermilk_min = 0
        mothermilk_result_right = mothermilk_pattern_right.match(l)
        if mothermilk_result_right:
            mothermilk_min = mothermilk_min + int(mothermilk_result_right.group(1))
        mothermilk_result_left = mothermilk_pattern_left.match(l)
        if mothermilk_result_left:
            mothermilk_min = mothermilk_min + int(mothermilk_result_left.group(1))
        if mothermilk_min > 0:
            df = df.append({
                'date': date,
                'time': time,
                'mothermilk': mothermilk_min
            }, ignore_index=True)
        
        milk_result = milk_pattern.match(l)
        if milk_result:
            df = df.append({
                'date': date,
                'time': time,
                'milk': int(milk_result.group(1))
            }, ignore_index=True)
        
        expressedmilk_result = expressedmilk_pattern.match(l)
        if expressedmilk_result:
            df = df.append({
                'date': date,
                'time': time,
                'expressedmilk': int(expressedmilk_result.group(1))
            }, ignore_index=True)
        
        small_result = small_pattern.match(l)
        if small_result:
            df = df.append({
                'date': date,
                'time': time,
                'small': 1
            }, ignore_index=True)
        
        big_result = big_pattern.match(l)
        if big_result:
            df = df.append({
                'date': date,
                'time': time,
                'big': 1
            }, ignore_index=True)
        
        sleep_result_in = sleep_pattern_in.match(l)
        if sleep_result_in:
            df = df.append({
                'date': date,
                'time': time,
                'sleep': 1
            }, ignore_index=True)
        sleep_result_out = sleep_pattern_out.match(l)
        if sleep_result_out:
            df = df.append({
                'date': date,
                'time': time,
                'sleep': 0
            }, ignore_index=True)

df

2020-10-01
2020-10-02
2020-10-03
2020-10-04
2020-10-05
2020-10-06
2020-10-07
2020-10-08
2020-10-09
2020-10-10
2020-10-11
2020-10-12
2020-10-13
2020-10-14
2020-10-15
2020-10-16
2020-10-17
2020-10-18
2020-10-19
2020-10-20
2020-10-21
2020-10-22
2020-10-23
2020-10-24
2020-10-25
2020-10-26
2020-10-27
2020-10-28
2020-10-29
2020-10-30
2020-10-31


,date,time,mothermilk,milk,expressedmilk,small,big,sleep
0,2020-10-01,01:00:00,11,NaN,NaN,NaN,NaN,NaN
1,2020-10-01,03:10:00,30,NaN,NaN,NaN,NaN,NaN
2,2020-10-01,06:45:00,NaN,NaN,NaN,NaN,NaN,0
3,2020-10-01,06:45:00,NaN,NaN,NaN,1,NaN,NaN
4,2020-10-01,06:45:00,23,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1273,2020-10-31,23:15:00,NaN,NaN,NaN,1,NaN,NaN
1274,2020-10-31,23:40:00,NaN,NaN,NaN,NaN,NaN,1
1275,2020-10-31,23:40:00,230,NaN,NaN,NaN,NaN,NaN
1276,2020-10-31,23:40:00,NaN,NaN,NaN,1,NaN,NaN


In [ ]:
df.to_csv(f'data/parsed/{target}.tsv', sep='\t')